<a href="https://colab.research.google.com/github/resh1998/VoterCRM_Backend/blob/main/BackEnd/Data_Collection/Main_Code_PDF_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Mounting Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/Images
!mkdir /content/drive/MyDrive/PDF_files
!mkdir /content/drive/MyDrive/Raw_CSV_files
!mkdir /content/drive/MyDrive/PDF_files

mkdir: cannot create directory ‘/content/drive/MyDrive/Images’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/PDF_files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/Raw_CSV_files’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/PDF_files’: File exists


#### Installing and Importing Packages

In [ ]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install Pillow
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (3,512 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 122531 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-eng_1%3

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (243 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 122709 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
from PIL import Image
from pytesseract import image_to_string
import pytesseract
import numpy as np
import pandas as pd
import PIL
from PIL import Image, ImageFilter, ImageEnhance
import cv2

import warnings
warnings.filterwarnings('ignore')

#### Initializing Dataframe

In [ ]:
# df = pd.DataFrame(columns = ['voter_id', 'name', 'relation_name', 'relation_type', 'house_number', 'age', 'gender'])



#### Funtion to input text in Excel(DONT RUN)

In [ ]:
# def write_to_dataframe(voter_id, name, relation_name, relation_type, house_number, age, gender):
  
#   global df

#   new_row = {'voter_id': voter_id, 'name': name, 'relation_name': relation_name, 
#              'relation_type': relation_type, 'house_number': house_number, 
#              'age': age, 'gender': gender}

#   df = df.append(new_row, ignore_index = True)



#### Function to get data from string (DONT RUN)

In [ ]:
# import re

# def get_data_from_string(text):
#   text = text.replace("\n", " ")
#   print(text)
#   voter_id =  ''
#   name = ''
#   relation_name = ''
#   relation_type = ''
#   house_number = ''
#   age = ''
#   gender = ''

#   if 'Husband' in text:

#     name = re.search(r'(?<=Name:).*(?=Husband\'s Name:)', text)

#     relation_name = re.search(r'(?<=Husband\'s Name:).*(?=House Number:)', text)
#     relation_type = 'H'
  
#   elif 'Wife' in text:
    
#     name = re.search(r'(?<=Name:).*(?=Wife\'s Name:)', text)

#     relation_name = re.search(r'(?<=Wife\'s Name:).*(?=House Number:)', text)
    
#     relation_type = 'W'
  
#   elif 'Father' in text:
    
#     name = re.search(r'(?<=Name:).*(?=Father\'s Name:)', text)
    
#     relation_name = re.search(r'(?<=Father\'s Name:).*(?=House Number:)', text)
    
#     relation_type = 'F'
  
#   house_number = re.search(r'(?<=House Number:).*(?=Age:)', text)

#   age = re.search(r'(?<=Age:).*(?=Gender:)', text)

#   gender = re.search(r'(?<=Gender:).*(?=)', text)

#   print(name, '\n', relation_name, '\n', house_number, '\n', age, '\n', gender, '\n')

#   write_to_dataframe(voter_id, name, relation_name, relation_type, house_number, age, gender)





#### Function to Get Single Voter

In [ ]:
import pandas as pd

def crop_page(contours, y_min, page_no):

  # read image from folder
  page_name = "/content/drive/MyDrive/Images/Page_" + str(page_no) + ".jpg"
  im = cv2.imread(page_name)

  # getting width of the image
  (h, w, s) = im.shape

  # converting the image to grayscale and binarizing it
  im = cv2.bilateralFilter(im,9,75,75)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 15)

  im = Image.fromarray(im)

  # getting the header of the page
  header_crop = (0, 0, w, y_min - 2)
  header_crop_im = im.crop(header_crop)

  text_header = image_to_string(header_crop_im, config = '--psm 6')

  # looping through each voter
  voter_shift = (-5, -52, 228, 5)
  voter_id_shift = (-600, 0, 0, 265)
  voter_no_shift = (-20, -3, 686, 294)

  for contour in contours:
    (x, y, w, h) = cv2.boundingRect(contour)
    voter_crop = tuple(np.subtract((x, y, x+w, y+h), voter_shift))
    voter_crop_im = im.crop(voter_crop)

    voter_id_crop = tuple(np.subtract((x, y, x+w, y+h), voter_id_shift))
    voter_id_crop_im = im.crop(voter_id_crop)

    voter_no_crop = tuple(np.subtract((x, y, x+w, y+h), voter_no_shift))
    voter_no_crop_im = im.crop(voter_no_crop)
    
    text_voter =  image_to_string(voter_crop_im, config = '--psm 6')
    text_voter_id =  image_to_string(voter_id_crop_im, config = '--psm 6')
    text_voter_no =  image_to_string(voter_no_crop_im, config = '--psm 6')

    # get_data_from_string(text_voter)

    # appending to dataframe
    new_row = {'voter_no': text_voter_no, 'voter_id': text_voter_id,'text': text_voter, 'header_section': text_header}

    global df
    df = df.append(new_row, ignore_index = True)


#### Contouring the Image

In [ ]:
import math
def contour_page(im):

  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  gray = cv2.medianBlur(gray, 5)
  gray = cv2.GaussianBlur(gray, (21, 21), 1)

  gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
  dilated = cv2.dilate(gray, kernel, iterations=1)

  eroded = cv2.erode(dilated, kernel, iterations=1)

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  morph = cv2.morphologyEx(eroded, cv2.MORPH_CLOSE, kernel, iterations=3)

  contours, hierarchy = cv2.findContours(eroded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = [c for c in contours if cv2.contourArea(c) > 100000 and cv2.contourArea(c) < 600000]
  
  y_min = math.inf

  for contour in contours:
      (x, y, w, h) = cv2.boundingRect(contour)  
      if y < y_min:
        y_min = y
      # cv2.rectangle(im, (x, y), (x + w, y + h), (0, 255, 0), 2)
  
  #im = cv2.bilateralFilter(im,9,75,75)
  #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  #im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 15)

  return contours, y_min

#### Converting PDF pages to single Image Page

In [ ]:
from pdf2image import convert_from_path

def convert_pdf_to_images(filename):
  pdfs = "/content/drive/MyDrive/PDF_files/" + filename + ".pdf"
  pages = convert_from_path(pdfs, 350)
  pdf_len = len(pages)

  for i in range(2, pdf_len-1):
    image_name = "/content/drive/MyDrive/Images/Page_" + str(i+1) + ".jpg"
    pages[i].save(image_name, "JPEG")
  
  return pdf_len


#### Read and convert each image

In [ ]:
## page_count = len(pages)

def read_and_convert_image(pdf_len):
# read image from directory
  for i in range(3, pdf_len):
    image_name = "/content/drive/MyDrive/Images/Page_" + str(i) + ".jpg"
    im = cv2.imread(image_name)

    contours, y_min = contour_page(im)

    crop_page(contours, y_min, i)
    

#### Main function and Reading PDF files

In [ ]:
df = pd.DataFrame(columns = ['voter_no', 'voter_id','text', 'header_section'])

In [ ]:
from pathlib import Path

pdf_files = Path("/content/drive/MyDrive/PDF_files").glob("*.pdf")

for pdf in pdf_files:

  filename = pdf.stem
  pdf_len = convert_pdf_to_images(filename)
  read_and_convert_image(pdf_len)
  
  csv_path = '/content/drive/MyDrive/Raw_CSV_files/' + filename + '.csv'
  df.to_csv(csv_path)
  
  !rm -rf /content/drive/MyDrive/Images
  
  !mkdir /content/drive/MyDrive/Images

  
  df = df[0:0]

